In [1]:
import re
import os
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
def fetch(url):
    fail = True
    while fail == True:
        time.sleep(1)
        result = requests.get(url)
        status_code = result.status_code
        if status_code == 200:
            content = result.content
            soup = BeautifulSoup(result.content, "html.parser")
            fail = False
            return soup
        else:
            print("Going to sleep")
            time.sleep(2)
            fail = True

In [3]:
def get_categories():
    top_category_url = "https://se.trustpilot.com/categories"
    category_soup = fetch(top_category_url)
    tags = category_soup.find("script", {"id":"__NEXT_DATA__"})
    data = json.loads(tags.text)
    tree = data["props"]
    all_categories = []
    cats = tree["pageProps"]["categories"]
    for category in cats:
        all_categories.append(category["categoryId"])
    category_urls = []
    for cat in all_categories:
        category_url = top_category_url + "/" + cat
        category_urls.append(category_url) # All category URLs collected from site
    return category_urls

In [4]:
categories = get_categories()

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
print(categories)

In [ ]:
data = []

if not os.path.exists("data"):
    os.makedir("data")

with open("ratings.csv","wb") as csv_file:
    for category in categories[0:1]:
        category_dir = category.split("/")
        category_dir = category_dir[len(category_dir)-1]
        if not os.path.exists("data/" + category_dir):
            os.mkdir("data/" + category_dir)

        category_specific_companies = []
        category_soup = fetch(category)
        category_tags = category_soup.find("script", {"id":"__NEXT_DATA__"})
        category_data = json.loads(category_tags.text)
        # Category is for example https://se.trustpilot.com/categories/animals_pets
        n_pages = category_data["props"]["pageProps"]["seoData"]["maxPages"]
        for i in range(1, n_pages):
            category_page = category + "?page=" + str(i)
            company_soup = fetch(category_page)
            company_tags = company_soup.find("script", {"id":"__NEXT_DATA__"})
            company_data = json.loads(company_tags.text)
            try: 

                # Collect company names
                company_tree = company_data["props"]["pageProps"]["businessUnits"]
                for k, v in company_tree.items():
                    if type(v) == list:
                        for j in v:
                            company_id = j["identifyingName"]
                            if company_id not in category_specific_companies:
                                category_specific_companies.append(company_id)
            except AttributeError:
                print("COMPANY ERROR 1! There was an error with this company:" + company_id)
            i += 1
        for c in category_specific_companies:
            try:
                # Collect company data
                if not os.path.exists("data/" + category_dir + "/" + c):
                    os.mkdir("data/" + category_dir + "/" + c)
                count_soup = fetch("https://se.trustpilot.com/review/" + c)
                count_tags = count_soup.find("script", {"id":"__NEXT_DATA__"})
                count_data = json.loads(count_tags.text)
                count_tree = count_data["props"]
                count_pages = count_tree["pageProps"]["filters"]["pagination"]["totalPages"]
            except AttributeError:
                print("COMPANY ERROR 2! There was an error with this company:" + c)
            for page in range(1, count_pages):
                try:
                    review_soup = fetch("https://se.trustpilot.com/review/" + c + "?page=" + str(page) +"&stars=3") # Specify rating if important here
                    review_tags = review_soup.find("script", {"id":"__NEXT_DATA__"})
                    review_data = json.loads(review_tags.text)
                    review_tree = review_data["props"]["pageProps"]
                    review_tree = review_tree["reviews"]
                    review = review_tree[0]["text"]
                    rating = review_tree[0]["rating"]
                    if rating == 3:
                        #data.append((review, rating, category, c))
                        #data = [rating, review, c, category]
                        data = "\"" + rating + "\"," + str(review) + "," + c + "," + category
                        csv_file.write(data)
                except AttributeError:
                    print("PAGE ERROR! There was an error on this page: " + "https://se.trustpilot.com/review/" + c + "?page=" + str(page))
        

In [ ]:
df = pd.DataFrame(data, columns=["text", "rating", "category", "company"])

In [ ]:
df

In [ ]:
data

In [ ]:
review = "Kanon bra service"
rating = 3
company "made_up_company"

with open("ratings.csv","a") as csv_file:
    csv_file.write(data)